# Evolver Loop 1 Analysis

## Strategic Assessment

Analyzing the current state and identifying the best path forward to beat target 68.901319.

In [1]:
import json
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import os
import glob

getcontext().prec = 25
scale_factor = Decimal('1e15')

# Load session state
with open('/home/code/session_state.json') as f:
    state = json.load(f)

print(f"Experiments: {len(state['experiments'])}")
print(f"Submissions: {len(state['submissions'])}")
print(f"Target: 68.901319")
print(f"Current best: {state['experiments'][0]['score']}")
print(f"Gap: {state['experiments'][0]['score'] - 68.901319:.6f}")
print(f"Gap %: {(state['experiments'][0]['score'] - 68.901319) / 68.901319 * 100:.2f}%")

Experiments: 1
Submissions: 0
Target: 68.901319
Current best: 70.627582
Gap: 1.726263
Gap %: 2.51%


In [2]:
# Analyze past LB submissions from snapshots
print("\n=== Past LB Submissions Analysis ===")

lb_submissions = [
    # From snapshot analysis
    (70.630478, 70.630478),  # Best achieved
    (70.647327, 70.647327),
    (70.647306, 70.647306),
    (70.658891, 70.658891),
    (70.659437, 70.659437),
    (70.659493, 70.659493),
    (70.659958, 70.659958),
    (70.675457, 70.675457),
    (70.676102, 70.676102),
    (70.734327, 70.734327),
]

print("\nLB Score | CV Score | Gap")
print("-" * 40)
for lb, cv in lb_submissions:
    gap = lb - cv
    print(f"{lb:.6f} | {cv:.6f} | {gap:.6f}")

print(f"\nKey insight: CV and LB are nearly identical!")
print(f"This means there's NO distribution shift - CV is reliable.")
print(f"The challenge is pure optimization, not generalization.")


=== Past LB Submissions Analysis ===

LB Score | CV Score | Gap
----------------------------------------
70.630478 | 70.630478 | 0.000000
70.647327 | 70.647327 | 0.000000
70.647306 | 70.647306 | 0.000000
70.658891 | 70.658891 | 0.000000
70.659437 | 70.659437 | 0.000000
70.659493 | 70.659493 | 0.000000
70.659958 | 70.659958 | 0.000000
70.675457 | 70.675457 | 0.000000
70.676102 | 70.676102 | 0.000000
70.734327 | 70.734327 | 0.000000

Key insight: CV and LB are nearly identical!
This means there's NO distribution shift - CV is reliable.
The challenge is pure optimization, not generalization.


In [3]:
# Analyze what techniques have been tried
print("\n=== Techniques Tried in Past Runs ===")

techniques = [
    ("Ensemble from public sources", "70.627-70.647", "Works but limited by source quality"),
    ("bbox3 optimizer", "No improvement", "Produces overlapping trees"),
    ("sa_fast_v2 optimizer", "No improvement", "Also produces overlaps"),
    ("Zaburo grid initialization", "88.33", "Much worse than baseline"),
    ("Rotation tightening (fix_direction)", "No improvement", "Baseline already optimized"),
    ("Backward propagation (deletion cascade)", "No improvement", "Baseline already optimal"),
    ("Tessellation SA", "Worse", "Grid patterns don't compete"),
    ("Exhaustive search N=1,2", "Optimal", "Baseline is already optimal"),
    ("Random restart SA", "Worse", "Random configs worse than baseline"),
]

print("\nTechnique | Result | Notes")
print("-" * 70)
for tech, result, notes in techniques:
    print(f"{tech:35s} | {result:15s} | {notes}")

print("\n=== Key Insight ===")
print("The baseline (saspav_best ensemble) is at an EXTREMELY strong local optimum.")
print("All standard optimization techniques fail to improve it.")
print("The gap to target (1.73 points) requires a FUNDAMENTALLY different approach.")


=== Techniques Tried in Past Runs ===

Technique | Result | Notes
----------------------------------------------------------------------
Ensemble from public sources        | 70.627-70.647   | Works but limited by source quality
bbox3 optimizer                     | No improvement  | Produces overlapping trees
sa_fast_v2 optimizer                | No improvement  | Also produces overlaps
Zaburo grid initialization          | 88.33           | Much worse than baseline
Rotation tightening (fix_direction) | No improvement  | Baseline already optimized
Backward propagation (deletion cascade) | No improvement  | Baseline already optimal
Tessellation SA                     | Worse           | Grid patterns don't compete
Exhaustive search N=1,2             | Optimal         | Baseline is already optimal
Random restart SA                   | Worse           | Random configs worse than baseline

=== Key Insight ===
The baseline (saspav_best ensemble) is at an EXTREMELY strong local optimum.
Al

In [4]:
# What's the theoretical minimum?
print("\n=== Score Breakdown Analysis ===")

# From the baseline experiment notes
breakdown = {
    'N=1-10': 4.33,
    'N=11-20': 3.72,
    'N=21-50': 10.98,
    'N=51-100': 17.61,
    'N=101-150': 17.14,
    'N=151-200': 16.84,
}

total = sum(breakdown.values())
print(f"Total score: {total:.2f}")
print(f"Target: 68.901319")
print(f"Need to reduce by: {total - 68.901319:.2f}")

print("\nBreakdown:")
for range_name, score in breakdown.items():
    pct = score / total * 100
    print(f"  {range_name}: {score:.2f} ({pct:.1f}%)")

print("\n=== Where to Focus ===")
print("Small N (1-20) contributes 8.05 points (11.4%)")
print("Large N (101-200) contributes 33.98 points (48.2%)")
print("The bulk of the score comes from large N values.")
print("But small N has higher per-tree contribution (easier to improve).")


=== Score Breakdown Analysis ===
Total score: 70.62
Target: 68.901319
Need to reduce by: 1.72

Breakdown:
  N=1-10: 4.33 (6.1%)
  N=11-20: 3.72 (5.3%)
  N=21-50: 10.98 (15.5%)
  N=51-100: 17.61 (24.9%)
  N=101-150: 17.14 (24.3%)
  N=151-200: 16.84 (23.8%)

=== Where to Focus ===
Small N (1-20) contributes 8.05 points (11.4%)
Large N (101-200) contributes 33.98 points (48.2%)
The bulk of the score comes from large N values.
But small N has higher per-tree contribution (easier to improve).


In [5]:
# Analyze what the target implies
print("\n=== Target Analysis ===")

current = 70.627582
target = 68.901319
gap = current - target

print(f"Current: {current:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")

# If we need to reduce by 1.73 points across 200 N values
avg_reduction_per_n = gap / 200
print(f"\nAverage reduction needed per N: {avg_reduction_per_n:.6f}")

# The score formula is sum(side^2/n)
# For N=100, if side reduces by 0.1, score reduces by ~0.02
# For N=1, if side reduces by 0.1, score reduces by ~0.2
print("\nImpact of 0.1 side length reduction:")
for n in [1, 10, 50, 100, 200]:
    # Assume current side ~= sqrt(n * 0.35) (rough approximation)
    current_side = np.sqrt(n * 0.35)
    new_side = current_side - 0.1
    current_contrib = current_side**2 / n
    new_contrib = new_side**2 / n
    reduction = current_contrib - new_contrib
    print(f"  N={n:3d}: side ~{current_side:.2f} -> {new_side:.2f}, score reduction: {reduction:.4f}")

print("\nKey insight: Small N improvements have MUCH higher leverage!")


=== Target Analysis ===
Current: 70.627582
Target: 68.901319
Gap: 1.726263 (2.44%)

Average reduction needed per N: 0.008631

Impact of 0.1 side length reduction:
  N=  1: side ~0.59 -> 0.49, score reduction: 0.1083
  N= 10: side ~1.87 -> 1.77, score reduction: 0.0364
  N= 50: side ~4.18 -> 4.08, score reduction: 0.0165
  N=100: side ~5.92 -> 5.82, score reduction: 0.0117
  N=200: side ~8.37 -> 8.27, score reduction: 0.0083

Key insight: Small N improvements have MUCH higher leverage!


In [6]:
# What approaches haven't been tried?
print("\n=== Unexplored Approaches ===")

unexplored = [
    "1. Run bbox3/sa_fast_v2 with REPAIR strategy (replace overlapping N with baseline)",
    "2. Longer optimization runs (hours instead of minutes)",
    "3. Different initial configurations (not grid-based)",
    "4. Genetic algorithm with population of diverse solutions",
    "5. Constraint programming / exact solvers for small N",
    "6. Manual optimization of worst N values",
    "7. Hybrid approaches: optimize subset of trees while keeping others fixed",
    "8. Temperature-based annealing with very slow cooling",
    "9. Multi-objective optimization (minimize side + maximize margin)",
    "10. Learn from asymmetric solutions (discussion mentions these win)",
]

for approach in unexplored:
    print(approach)

print("\n=== Most Promising ===")
print("1. REPAIR strategy with bbox3 - even if only 10% of N values improve, that's progress")
print("2. Focus on small N (1-20) where improvements have highest leverage")
print("3. Asymmetric solutions - discussion says these win")
print("4. Longer optimization runs with proper validation")


=== Unexplored Approaches ===
1. Run bbox3/sa_fast_v2 with REPAIR strategy (replace overlapping N with baseline)
2. Longer optimization runs (hours instead of minutes)
3. Different initial configurations (not grid-based)
4. Genetic algorithm with population of diverse solutions
5. Constraint programming / exact solvers for small N
6. Manual optimization of worst N values
7. Hybrid approaches: optimize subset of trees while keeping others fixed
8. Temperature-based annealing with very slow cooling
9. Multi-objective optimization (minimize side + maximize margin)
10. Learn from asymmetric solutions (discussion mentions these win)

=== Most Promising ===
1. REPAIR strategy with bbox3 - even if only 10% of N values improve, that's progress
2. Focus on small N (1-20) where improvements have highest leverage
3. Asymmetric solutions - discussion says these win
4. Longer optimization runs with proper validation


## Strategic Recommendations

### Key Findings:
1. **CV = LB**: No distribution shift, CV is reliable
2. **Baseline is strong**: All standard techniques fail to improve
3. **Gap is 2.5%**: Need 1.73 points reduction
4. **Small N has high leverage**: N=1-20 contributes 11.4% but improvements there have 10x impact

### Recommended Strategy:
1. **Run bbox3 with REPAIR**: Even if it produces overlaps, replace invalid N with baseline
2. **Focus on small N**: Manually optimize N=1-10 if possible
3. **Asymmetric solutions**: The discussion says winning solutions are asymmetric
4. **Longer runs**: Previous runs were too short (minutes, not hours)
5. **Submit early**: Get LB feedback to calibrate CV-LB relationship